# Assignment 5: Population Analysis
Author: Zoe McNamara Harlowe

## Imports

In [2]:
# Dataframes/Data manipulation
import pandas as pd

# Numerical computing
import numpy as np

# Regular expressions
import regex as re

## Part 1 
*Write a Jupyter notebook that analyses the differences between the sexes by age in Ireland.*
- *Weighted mean age (by sex)*
- *The difference between the sexes by age*

First, I read in the dataset and chose which columns I would be using

In [3]:
FILENAME="cso-populationbysex.csv"
DATADIR=r"C:\Users\ZMH\OneDrive\Desktop\PFDA\data\\" # I got a FileNotFoundError when I used "../../data/"
FULLPATH =  DATADIR + FILENAME

# Read in the CSV file
df = pd.read_csv(FULLPATH)

# Filter to only include rows where "Administrative Counties" is "Ireland"
df = df[df["Administrative Counties"] == "Ireland"]

# Create a list of columns to drop
drop_col_list = ["Statistic Label","CensusYear","Administrative Counties","UNIT"]

# Drop columns
df.drop(columns=drop_col_list, inplace=True)

# Have a look
df.head()

,Sex,Single Year of Age,VALUE
0,Male,All ages,2544549
32,Male,Under 1 year,29610
64,Male,1 year,28875
96,Male,2 years,30236
128,Male,3 years,31001


Then I removed the "All ages" from the dataset as I want to deal with each age individually.

I tidied up the ages, replacing "Under 1 year" to just "0" and also removing the "years old" from the dataset.

I also changed the ages to integers as they were strings

In [3]:
# Remove "All ages" from the dataset
df = df[df["Single Year of Age"] != "All ages"]

# I was getting errors when trying to tidy up the ages, so I changed the datatype to string first
df['Single Year of Age']=df['Single Year of Age'].astype('str')
df["VALUE"] = df["VALUE"].astype('str')

# Tidy up ages
df["Single Year of Age"] = df["Single Year of Age"].str.replace(r'Under 1 year', '0')
df["Single Year of Age"] = df["Single Year of Age"].str.replace(r'\D', '', regex=True)

# Change ages to int
df['Single Year of Age']=df['Single Year of Age'].astype('int64')
df["VALUE"] = df["VALUE"].astype('int64')

# Show
df.head()


,Sex,Single Year of Age,VALUE
32,Male,0,29610
64,Male,1,28875
96,Male,2,30236
128,Male,3,31001
160,Male,4,31686


I then created a pivot table with the average age by sex.

Then I wrote out the cleaned up pivot tables to my computer

In [4]:
# Create pivot table with different columns for male and female
df_pivot = pd.pivot_table(df, values="VALUE", index="Single Year of Age", columns="Sex", aggfunc="sum")
print (df_pivot.head(3))

# write out the entire file to local machine
df_pivot.to_csv("population_for_analysis.csv")

Sex                 Female   Male
Single Year of Age               
0                    28186  29610
1                    27545  28875
2                    28974  30236


Now that I have cleaned up my dataframe, I can now do some analysis.

First I want to get the weighted mean age by sex

Weighted mean is sum(age*population at age) / sum (populations at age)

In [5]:
df_pivot

Sex,Female,Male
Single Year of Age,,
0,28186,29610
1,27545,28875
2,28974,30236
3,29483,31001
4,29819,31686
...,...,...
96,956,327
97,732,217
98,492,130


First I got the number of females and number of males

In [6]:
# Number of females
number_females = df_pivot["Female"].sum()
print(f"No. of females: {number_females}")

# Number of males
number_males = df_pivot["Male"].sum()
print(f"No. of males: {number_males}")

No. of females: 2604590
No. of males: 2544549


Then I got the cumulative ages for females and males

In [7]:
# Cumulative ages for females
cumages_female = df_pivot["Female"].mul(df_pivot.index, axis=0).sum()
print(cumages_female)

# Cumulative ages for males
cumages_male = df_pivot["Male"].mul(df_pivot.index, axis=0).sum()
print(cumages_male)

101422203
96029874


Now I can find the weighted mean for both sexes

In [8]:
# Female weighted mean age
w_mean_f = cumages_female / number_females
print(f"Weighted mean age for females in Ireland: {w_mean_f}")

# Male weighted mean age
w_mean_m = cumages_male / number_males
print(f"Weighted mean age for males in Ireland: {w_mean_m}")

Weighted mean age for females in Ireland: 38.9397958987787
Weighted mean age for males in Ireland: 37.7394477371039


The second task in Part 1 was: *Find the difference between the sexes by age.*

My assumption here was to show the population difference between males and females for each age group.

In [ ]:
# I created a new column in df_pivot called "pop_diff"
# I store the absolute value of the population difference in this column
df_pivot["pop_diff"] = abs(df_pivot["Female"] - df_pivot["Male"]) # abs to ensure positive number

# Show
df_pivot

Sex,Female,Male,pop_diff
Single Year of Age,,,
0,28186,29610,1424
1,27545,28875,1330
2,28974,30236,1262
3,29483,31001,1518
4,29819,31686,1867
...,...,...,...
96,956,327,629
97,732,217,515
98,492,130,362


## Part 2
*In the same notebook, make a variable that stores an age (say 35).*

*Write that code that would group the people within 5 years of that age together, into one age group*

*Calculate the population difference between the sexes in that age group.*

I created the variable `age` that stored the age 35.

Then I used the existing pivot table `df_pivot` and used indexing to include those within 5 years of age 35. (i.e. 30-40)

In [35]:
# Create variable that stores an age
age = 35

# Create age group within 5 years of that age
age_group = df_pivot[age-5:age+6] 

# Show
age_group


Sex,Female,Male,pop_diff
Single Year of Age,,,
30,32841,30858,1983
31,33710,32237,1473
32,34382,32413,1969
33,34489,31888,2601
34,36284,33121,3163
35,37940,34695,3245
36,39030,35828,3202
37,39193,36427,2766
38,40902,37513,3389


Then I used the `cumsum()` function to find the cumulative ages for females and males in that age group.

I subtracted the final values in the female and male columns from one another to get the population difference, using the absolute value to ensure the resulting number was positive. 

In [29]:
# Create sexes variable
sexes = df_pivot.columns

# Calculate cumulative ages for females and males
cumages_female = age_group["Female"].cumsum()
cumages_male = age_group["Male"].cumsum()
    
# Calculate population difference between sexes in the age group
pop_diff = abs(cumages_female.iloc[-1] - cumages_male.iloc[-1]) # absolute value to ensure positive number

I then made my print statement say whether there were more females or males using 2 if statements.

This probably could have been done in an easier way but it's what made sense to me

In [30]:
# If statement to determine if females or males are more numerous
if cumages_female.iloc[-1] > cumages_male.iloc[-1]:
    more_numerous = "females"
else:
    more_numerous = "males"

# If statement to show which sex is less numerous
if cumages_female.iloc[-1] < cumages_male.iloc[-1]:
    less_numerous = "females"
else:
    less_numerous = "males"

print(f"Population difference in Ireland between males and females of {age-5} and {age+5} years old:")
print(f"There are {pop_diff} more {more_numerous} than {less_numerous}.")

Population difference in Ireland between males and females of 30 and 40 years old:
There are 30476 more females than males.


# Part 3

*Write the code that would work out which region in Ireland has the biggest population difference between the sexes in that age group*

To do this, I need to go back to the original dataset from the CSO.

In [45]:
# Read in the CSV file again
df = pd.read_csv(FULLPATH)

# Create a list of columns to drop - keeping Administrative Counties this time
drop_col_list = ["Statistic Label","CensusYear","UNIT"]

# Drop columns
df.drop(columns=drop_col_list, inplace=True)

I then cleaned the dataset again, same as above in Part 1.

In [54]:
# Remove "All ages" from the dataset
df = df[df["Single Year of Age"] != "All ages"]

# Remove "Ireland" from the dataset
df = df[df["Administrative Counties"] != "Ireland"]

# Tidy up ages again (copied from the beginning of Part 1)
# I was getting errors when trying to tidy up the ages, so I changed the datatype to string first
df['Single Year of Age']=df['Single Year of Age'].astype('str')
df["VALUE"] = df["VALUE"].astype('str')

# Change 'Under 1 year' to '0' and remove non-digit characters
df["Single Year of Age"] = df["Single Year of Age"].str.replace(r'Under 1 year', '0')
df["Single Year of Age"] = df["Single Year of Age"].str.replace(r'\D', '', regex=True)

# Change ages to int
df['Single Year of Age']=df['Single Year of Age'].astype('int64')
df["VALUE"] = df["VALUE"].astype('int64')

I wasn't sure what the most efficient way to create the age groups was.

I asked ChatGPT (See conversation here: https://chatgpt.com/share/690a49ad-fd14-800c-9820-94e09dfc9102)

It got me to create variables `bins` and `labels` to make age groups 0-9, 10-19, 20-29, etc.

In [60]:
# Define bins and labels
bins = list(range(0, 101, 10))  # up to 100 (adjust if necessary)
labels = [f"{i}-{i+9}" for i in bins[:-1]]

# Create Age Group column
df["Age Group"] = pd.cut(df['Single Year of Age'], bins=bins, labels=labels, right=False)

In the same conversation with ChatGPT, I learned how to use the `groupby()` function to better organise the layout of the dataset.

Now I can easier see the regions and populations per sex by age group in each region.

In [ ]:
# Create grouped dataframe
grouped = (
    df.groupby(["Administrative Counties", "Age Group", "Sex"])["VALUE"] # Aggregate VALUE column
      .sum() # Sum the values
      .reset_index("Age Group" 
))
print(grouped.head())

                               Age Group  VALUE
Administrative Counties Sex                    
Carlow County Council   Female       0-9   3762
                        Male         0-9   3863
                        Female     10-19   4401
                        Male       10-19   4609
                        Female     20-29   3407


C:\Users\ZMH\AppData\Local\Temp\ipykernel_45304\2868066007.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["Administrative Counties", "Age Group", "Sex"])["VALUE"]


I used the grouped dataframe to create my pivot table.

Then I created another column `Sex Difference` to show the difference in numbers of males vs females per county

In [82]:
# Create pivot table
pivot = grouped.pivot_table( # use the grouped dataframe
    index=["Administrative Counties", "Age Group"], # index
    columns='Sex', # columns
    values='VALUE', # population values
).reset_index()

# Create column 'Sex Difference' that stores absolute difference between male and female populations
pivot['Sex Difference'] = (pivot['Male'] - pivot['Female']).abs()

# Convert columns to int64 as they were coming up as floats
pivot['Male'] = pivot['Male'].astype('int64')
pivot['Female'] = pivot['Female'].astype('int64')
pivot['Sex Difference'] = pivot['Sex Difference'].astype('int64')

# Show
pivot.head(10)

C:\Users\ZMH\AppData\Local\Temp\ipykernel_45304\3041614110.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = grouped.pivot_table( # use the grouped dataframe


Sex,Administrative Counties,Age Group,Female,Male,Sex Difference
0,Carlow County Council,0-9,3762,3863,101
1,Carlow County Council,10-19,4401,4609,208
2,Carlow County Council,20-29,3407,3666,259
3,Carlow County Council,30-39,4287,3957,330
4,Carlow County Council,40-49,4788,4671,117
5,Carlow County Council,50-59,3983,3948,35
6,Carlow County Council,60-69,3154,3079,75
7,Carlow County Council,70-79,2162,2158,4
8,Carlow County Council,80-89,1002,793,209
9,Carlow County Council,90-99,197,78,119


Now that I have my pivot table, I need to find the county with the biggest age difference in the 30-39 category.

I decided to use the pandas `.sort_value()` function to find the biggest value (See: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html)

In [77]:
# Filter for the desired age group
age_filter = pivot[pivot['Age Group'] == '30-39']

# Sort by Sex Difference descending
age_filter_sorted = age_filter.sort_values('Sex Difference', ascending=False)

# See the top region
print(age_filter_sorted.head())


Sex      Administrative Counties Age Group  Female   Male  Sex Difference
83         Fingal County Council     30-39   25961  23177            2784
43           Cork County Council     30-39   23487  20962            2525
253  South Dublin County Council     30-39   23539  21062            2477
123       Kildare County Council     30-39   18326  16581            1745
203         Meath County Council     30-39   15788  14194            1594


In [90]:
# Print the region with the biggest difference
print(f"The county with the biggest population difference between males and females aged 30-39 is:\n{age_filter_sorted.iloc[0]['Administrative Counties']} with a difference of {age_filter_sorted.iloc[0]['Sex Difference']}.")

The county with the biggest population difference between males and females aged 30-39 is:
Fingal County Council with a difference of 2784.


# End